# Talmud Lab, Word Translation Project

A demonstration of the latest methods developed for the Talmud word-to-Jastrow project, which aims to map every word in the Talmud to a single entry (or ranked list of entries) in the Jastrow Talmudic dictionary.

In [1]:
from utils.texts import *
from utils.deconstruct2 import *
from utils.jastrow import *
from utils.respell import *

In [2]:
import re
from string import punctuation

import numpy as np
import pandas as pd
import random as rd

## Download a masekhet

`get_masekhet` downloads the full Hebrew text of a masekhet and tags the Steinsaltz "chunks" based on whether they are mishnayot or gemaras.

In [3]:
ber = Masekhet('Berakhot')

In [5]:
ber[:5]

[[['מֵאֵימָתַי',
   'קוֹרִין',
   'אֶת',
   'שְׁמַע',
   'בָּעֲרָבִין',
   '?',
   'מִשָּׁעָה',
   'שֶׁהַכֹּהֲנִים',
   'נִכְנָסִים',
   'לֶאֱכוֹל',
   'בִּתְרוּמָתָן',
   '.',
   'עַד',
   'סוֹף',
   'הָאַשְׁמוּרָה',
   'הָרִאשׁוֹנָה',
   '.',
   'דִּבְרֵי',
   'רַבִּי',
   'אֱלִיעֶזֶר',
   '.'],
  ['וַחֲכָמִים', 'אוֹמְרִים', ':', 'עַד', 'חֲצוֹת', '.'],
  ['רַבָּן',
   'גַּמְלִיאֵל',
   'אוֹמֵר',
   'עַד',
   'שֶׁיַּעֲלֶה',
   'עַמּוּד',
   'הַשַּׁחַר',
   '.'],
  ['מַעֲשֶׂה',
   'וּבָאוּ',
   'בָנָיו',
   'מִבֵּית',
   'הַמִּשְׁתֶּה',
   ',',
   'אָמְרוּ',
   'לוֹ',
   ':',
   'לֹא',
   'קָרִינוּ',
   'אֶת',
   'שְׁמַע',
   ',',
   'אָמַר',
   'לָהֶם',
   ':',
   'אִם',
   'לֹא',
   'עָלָה',
   'עַמּוּד',
   'הַשַּׁחַר',
   'חַיָּיבִין',
   'אַתֶּם',
   'לִקְרוֹת',
   '.',
   'וְלֹא',
   'זוֹ',
   'בִּלְבַד',
   'אָמְרוּ',
   ',',
   'אֶלָּא',
   'כָּל',
   'מַה',
   'שֶּׁאָמְרוּ',
   'חֲכָמִים',
   '״',
   'עַד',
   'חֲצוֹת',
   '״',
   ',',
   'מִצְוָתָן',
   'עַד',
   'שֶׁיַּעֲלֶה

In [6]:
len(ber)

125

## Removing prefixes

`detach_prefix` removes all possible prefixes from a word based on the rules of nikkud to get its root.

In [4]:
r_page = rd.randrange(len(ber))
r_chunk = rd.randrange(len(ber[r_page]))
selected = ber[r_page][r_chunk]
print(' '.join(selected))
words = [i for i in selected if not (i in punctuation + '…׳״—')]

words

תָּנוּ רַבָּנַן : גְּרָף שֶׁל רְעִי וְעָבִיט שֶׁל מֵי רַגְלַיִם — אָסוּר לִקְרוֹת קְרִיאַת שְׁמַע כְּנֶגְדָּן , וְאַף עַל פִּי שֶׁאֵין בָּהֶן כְּלוּם . וּמֵי רַגְלַיִם עַצְמָן — עַד שֶׁיָּטִיל לְתוֹכָן מַיִם . וְכַמָּה יָטִיל לְתוֹכָן מַיִם — כָּל שֶׁהוּא . רַבִּי זַכַּאי אוֹמֵר : רְבִיעִית . בֵּין לִפְנֵי הַמִּטָּה בֵּין לְאַחַר הַמִּטָּה . רַבָּן שִׁמְעוֹן בֶּן גַּמְלִיאֵל אוֹמֵר : לְאַחַר הַמִּטָּה — קוֹרֵא , לִפְנֵי הַמִּטָּה — אֵינוֹ קוֹרֵא , אֲבָל מַרְחִיק הוּא אַרְבַּע אַמּוֹת וְקוֹרֵא . רַבִּי שִׁמְעוֹן בֶּן אֶלְעָזָר אוֹמֵר : אֲפִילּוּ בֵּית מֵאָה אַמָּה — לֹא יִקְרָא עַד שֶׁיּוֹצִיאֵם אוֹ שֶׁיַּנִּיחֵם תַּחַת הַמִּטָּה .


['תָּנוּ',
 'רַבָּנַן',
 'גְּרָף',
 'שֶׁל',
 'רְעִי',
 'וְעָבִיט',
 'שֶׁל',
 'מֵי',
 'רַגְלַיִם',
 'אָסוּר',
 'לִקְרוֹת',
 'קְרִיאַת',
 'שְׁמַע',
 'כְּנֶגְדָּן',
 'וְאַף',
 'עַל',
 'פִּי',
 'שֶׁאֵין',
 'בָּהֶן',
 'כְּלוּם',
 'וּמֵי',
 'רַגְלַיִם',
 'עַצְמָן',
 'עַד',
 'שֶׁיָּטִיל',
 'לְתוֹכָן',
 'מַיִם',
 'וְכַמָּה',
 'יָטִיל',
 'לְתוֹכָן',
 'מַיִם',
 'כָּל',
 'שֶׁהוּא',
 'רַבִּי',
 'זַכַּאי',
 'אוֹמֵר',
 'רְבִיעִית',
 'בֵּין',
 'לִפְנֵי',
 'הַמִּטָּה',
 'בֵּין',
 'לְאַחַר',
 'הַמִּטָּה',
 'רַבָּן',
 'שִׁמְעוֹן',
 'בֶּן',
 'גַּמְלִיאֵל',
 'אוֹמֵר',
 'לְאַחַר',
 'הַמִּטָּה',
 'קוֹרֵא',
 'לִפְנֵי',
 'הַמִּטָּה',
 'אֵינוֹ',
 'קוֹרֵא',
 'אֲבָל',
 'מַרְחִיק',
 'הוּא',
 'אַרְבַּע',
 'אַמּוֹת',
 'וְקוֹרֵא',
 'רַבִּי',
 'שִׁמְעוֹן',
 'בֶּן',
 'אֶלְעָזָר',
 'אוֹמֵר',
 'אֲפִילּוּ',
 'בֵּית',
 'מֵאָה',
 'אַמָּה',
 'לֹא',
 'יִקְרָא',
 'עַד',
 'שֶׁיּוֹצִיאֵם',
 'אוֹ',
 'שֶׁיַּנִּיחֵם',
 'תַּחַת',
 'הַמִּטָּה']

In [5]:
for w in words:
    print(w, end='\t')
    print(detach_prefix(w))

תָּנוּ	['תָּנוּ']
רַבָּנַן	['רַבָּנַן']
גְּרָף	['גְּרָף']
שֶׁל	['שֶׁל']
רְעִי	['רְעִי']
וְעָבִיט	['וְעָבִיט', 'עָבִיט']
שֶׁל	['שֶׁל']
מֵי	['מֵי']
רַגְלַיִם	['רַגְלַיִם']
אָסוּר	['אָסוּר']
לִקְרוֹת	['לִקְרוֹת', 'קְרוֹת']
קְרִיאַת	['קְרִיאַת']
שְׁמַע	['שְׁמַע']
כְּנֶגְדָּן	['כְּנֶגְדָּן', 'נֶגְדָּן']
וְאַף	['וְאַף', 'אַף']
עַל	['עַל']
פִּי	['פִּי']
שֶׁאֵין	['שֶׁאֵין', 'אֵין']
בָּהֶן	['בָּהֶן']
כְּלוּם	['כְּלוּם', 'לוּם']
וּמֵי	['וּמֵי', 'מֵי']
רַגְלַיִם	['רַגְלַיִם']
עַצְמָן	['עַצְמָן']
עַד	['עַד']
שֶׁיָּטִיל	['שֶׁיָּטִיל', 'יָּטִיל']
לְתוֹכָן	['לְתוֹכָן', 'תוֹכָן']
מַיִם	['מַיִם']
וְכַמָּה	['וְכַמָּה', 'כַמָּה']
יָטִיל	['יָטִיל']
לְתוֹכָן	['לְתוֹכָן', 'תוֹכָן']
מַיִם	['מַיִם']
כָּל	['כָּל']
שֶׁהוּא	['שֶׁהוּא', 'הוּא']
רַבִּי	['רַבִּי']
זַכַּאי	['זַכַּאי']
אוֹמֵר	['אוֹמֵר']
רְבִיעִית	['רְבִיעִית']
בֵּין	['בֵּין']
לִפְנֵי	['לִפְנֵי', 'פְנֵי']
הַמִּטָּה	['הַמִּטָּה', 'מִּטָּה']
בֵּין	['בֵּין']
לְאַחַר	['לְאַחַר', 'חַר', 'אַחַר']
הַמִּטָּה	['הַמִּטָּה', 'מִּטָּה']
רַבָּן	['רַבָּן']
שִׁמְעוֹ

## Getting roots

In [8]:
for w in words:
    print(w, end='\t')
    
    possibilities = detach_prefix(w)
    print(possibilities)
    
    print('Hebrew:')
    for p in possibilities:
        print(p + '\t' + str(hebrew_root(p)))
    
    print('Aramaic verb:')
    for p in possibilities:
        print(p + '\t' + str(aramaic_verb_root(p)))
    
    print('Aramaic noun:')
    for p in possibilities:
        print(p + '\t' + str(aramaic_noun_root(p)))
    
    print('\n\n')

תָּנוּ	['תָּנוּ']
Hebrew:
תָּנוּ	[('נָתַן', 'פועל', 'קל'), ('תִּנָּה', 'פועל', 'פיעל')]
Aramaic verb:
תָּנוּ	[('תְּנָא', 'Paal')]
Aramaic noun:
תָּנוּ	[]



רַבָּנַן	['רַבָּנַן']
Hebrew:
רַבָּנַן	[]
Aramaic verb:
רַבָּנַן	[]
Aramaic noun:
רַבָּנַן	['רַבָּנָא']



גְּרָף	['גְּרָף']
Hebrew:
גְּרָף	[('גְּרָף', 'שֵם'), ('גָּרַף', 'פועל', 'קל'), ('גְּרָף', 'שֵם'), ('גְרָף', 'סופית'), ('גֵּרֵף', 'פועל', 'פיעל'), ('גֹּרַף', 'פועל', 'פועל')]
Aramaic verb:
גְּרָף	[]
Aramaic noun:
גְּרָף	[]



שֶׁל	['שֶׁל']
Hebrew:
שֶׁל	[('שֶׁל', ''), ('שָׁל', 'שֵם'), ('נָשַׁל', 'פועל', 'קל')]
Aramaic verb:
שֶׁל	[]
Aramaic noun:
שֶׁל	[]



רְעִי	['רְעִי']
Hebrew:
רְעִי	[('רְעִי', 'שֵם'), ('רַע', 'תואר'), ('רֵעַ', 'שֵם'), ('רָעָה', 'פועל', 'קל'), ('רֵעַ', 'שֵם')]
Aramaic verb:
רְעִי	[('רְעָא', 'Paal')]
Aramaic noun:
רְעִי	[]



וְעָבִיט	['וְעָבִיט', 'עָבִיט']
Hebrew:
וְעָבִיט	[('עָבִיט', 'שֵם')]
עָבִיט	[('עָבִיט', 'שֵם')]
Aramaic verb:
וְעָבִיט	[]
עָבִיט	[]
Aramaic noun:
וְעָבִיט	[]
עָבִיט	[]



שֶׁל	['שֶׁל']
Heb

מִּטָּה	[]
Aramaic noun:
הַמִּטָּה	[]
מִּטָּה	[]



בֵּין	['בֵּין']
Hebrew:
בֵּין	[('בֵּין', 'מילת יחס'), ('בֵּין', 'תחילית'), ('יֶן', 'שֵם'), ('בִּיֵּן', 'פועל', 'פיעל')]
Aramaic verb:
בֵּין	[]
Aramaic noun:
בֵּין	[]



לְאַחַר	['לְאַחַר', 'חַר', 'אַחַר']
Hebrew:
לְאַחַר	[('לְאַחַר', 'מילת יחס'), ('אִחֵר', 'פועל', 'פיעל'), ('אַחֵר', 'תואר')]
חַר	[('חֹר', 'שֵם'), ('חֹר', 'שֵם')]
אַחַר	[('אַחַר', 'מילת יחס'), ('אַחֵר', 'תואר'), ('אִחֵר', 'פועל', 'פיעל'), ('אֻחַר', 'פועל', 'פועל')]
Aramaic verb:
לְאַחַר	[]
חַר	[('', 'Pael'), ('חַר', 'Paal')]
אַחַר	[]
Aramaic noun:
לְאַחַר	[]
חַר	[]
אַחַר	[]



הַמִּטָּה	['הַמִּטָּה', 'מִּטָּה']
Hebrew:
הַמִּטָּה	[('הֲמָטָה', 'שֵם'), ('מַטֶּה', 'שֵם'), ('הִטָּה', 'פועל', 'הפעיל'), ('מִטָּה', 'שֵם'), ('מָט', 'פועל', 'קל'), ('מֻטֶּה', 'תואר'), ('הֻטָּה', 'פועל', 'הופעל'), ('מֶטָא', 'תחילית'), ('מֻטָּה', 'שֵם')]
מִּטָּה	[('מַטֶּה', 'שֵם'), ('מַטָּה', 'תואר הפועל'), ('הִטָּה', 'פועל', 'הפעיל'), ('מֶטָא', 'תחילית'), ('מָט', 'פועל', 'קל'), ('מִטָּה', 'שֵם'), ('

הַמִּטָּה	[('הֲמָטָה', 'שֵם'), ('מַטֶּה', 'שֵם'), ('הִטָּה', 'פועל', 'הפעיל'), ('מִטָּה', 'שֵם'), ('מָט', 'פועל', 'קל'), ('מֻטֶּה', 'תואר'), ('הֻטָּה', 'פועל', 'הופעל'), ('מֶטָא', 'תחילית'), ('מֻטָּה', 'שֵם')]
מִּטָּה	[('מַטֶּה', 'שֵם'), ('מַטָּה', 'תואר הפועל'), ('הִטָּה', 'פועל', 'הפעיל'), ('מֶטָא', 'תחילית'), ('מָט', 'פועל', 'קל'), ('מִטָּה', 'שֵם'), ('מֻטֶּה', 'תואר'), ('הֻטָּה', 'פועל', 'הופעל'), ('מֻטָּה', 'שֵם')]
Aramaic verb:
הַמִּטָּה	[]
מִּטָּה	[]
Aramaic noun:
הַמִּטָּה	[]
מִּטָּה	[]





## Naive Jastrow Search

The following description has been pulled from the docs of the method used below:



`An extremely naive entry searcher, intended merely as a proof-of-concept, not for practical use.`

`The method uses naive lookup on different constructions of 'word' and returns the top N matching entries.`

`The "top N" are ranked by category:`

`1. In Jastrow directly.`

`2. Top entries from dicta nouns (since there are fewer nouns than verbs)`

`3. Top entries from dicta verbs`

`4. Top entries from Morfix.`

`The word is first put through respell.order_nikkud, and if more than one spelling is returned, the method returns`

`the top N for each spelling. Then its possible prefixes are removed, with the ranking choosing the shortest among`

`the possibilities as top rank.`

In [6]:
i = 1
for w in words:
    print(i, end='. ')
    print(w, end='\t')
    print(naive_top_n(w, 100))
    i += 1

1. תָּנוּ	['תְּנָא', 'תְּנֵי', 'נָתַן']
2. רַבָּנַן	['רַבָּן ²']
3. גְּרָף	['גָּרַף']
4. שֶׁל	['שֶׁל', 'נָשַׁל']
5. רְעִי	['רְעִי III', 'רָעָה', 'רֵעַ', 'רְעָא', 'רְעֵי', 'רְעִי ²', 'רְעִי', 'רַע II', 'רְעֵי ²', 'רַע III', 'רִיע', 'רַע I', 'רָעָה ²', 'רעי']
6. וְעָבִיט	['עָבִיט']
7. שֶׁל	['שֶׁל', 'נָשַׁל']
8. מֵי	['מִתְנָא', 'מִסְכֵּנוּ', 'מְשַׁח IV', 'מִצְרָאָה', 'מִנְיוֹמִי', 'מִכְלָה', 'מֵלַךְ', 'מִנְיַן', 'מִתגָּא', 'מִלְלָא', 'מֶלַח ²', 'מְנָן', 'מִלָּה', 'מִשְׁתִּי', 'מֵצַר ³', 'מִי I', 'מִי II', 'מִשְׁלָם', 'מִסְפְּק', 'מֵסֵב', 'מִנְיָן', 'מֵרַע', 'מֵרַד', 'מִ־', 'מִסְתְּיָא', 'מֵעֲלָא', 'מַלְיְתָא', 'מַיִם', 'מִסְּתָא', 'מַיָּא']
9. רַגְלַיִם	['רַגְלִי', 'רֶגֶל ²', 'רֶגֶל']
10. אָסוּר	['אֲסַר', 'אָסוּר', 'אִסּוּר', 'סָר ²', 'אִיסּוּר II', 'סָר', 'אָסַר']
11. לִקְרוֹת	['קָרָה', 'קַר', 'קרי ³', 'קרי ⁴']
12. קְרִיאַת	['קְרִיאָה']
13. שְׁמַע	['שָׁמַע', 'שֵׁמַע', 'שְׁמַע I', 'שְׁמַע II', 'שֵׁמַע ²']
14. כְּנֶגְדָּן	['נֶגֶד']
15. וְאַף	['אַף III', 'אַף II', 'אַף I']
16. עַל	['עֹל', '

## Hebrew verb root getter

In [9]:
i = 1
for w in words:
    print(str(i) + '. ' + w, end='\t')
    
    possibilities = detach_prefix(w)
    print(possibilities)
    
    for p in possibilities:
        heb_roots = hebrew_root(p)
        for r in heb_roots:
            if r[1] == 'פועל':
                print('\t', end='')
                print(str(r), end='\t')
                print(hebrew_shoresh(r[0], r[2]))
        
    i += 1

1. תָּנוּ	['תָּנוּ']
	('נָתַן', 'פועל', 'קל')	נתן
	('תִּנָּה', 'פועל', 'פיעל')	תנה
2. רַבָּנַן	['רַבָּנַן']
3. גְּרָף	['גְּרָף']
	('גָּרַף', 'פועל', 'קל')	גרף
	('גֵּרֵף', 'פועל', 'פיעל')	גרף
	('גֹּרַף', 'פועל', 'פועל')	גרף
4. שֶׁל	['שֶׁל']
	('נָשַׁל', 'פועל', 'קל')	נשל
5. רְעִי	['רְעִי']
	('רָעָה', 'פועל', 'קל')	רעה
6. וְעָבִיט	['וְעָבִיט', 'עָבִיט']
7. שֶׁל	['שֶׁל']
	('נָשַׁל', 'פועל', 'קל')	נשל
8. מֵי	['מֵי']
9. רַגְלַיִם	['רַגְלַיִם']
10. אָסוּר	['אָסוּר']
	('סָר', 'פועל', 'קל')	סר
	('אָסַר', 'פועל', 'קל')	אסר
	('אָסַר', 'פועל', 'קל')	אסר
11. לִקְרוֹת	['לִקְרוֹת', 'קְרוֹת']
	('קָרָה', 'פועל', 'קל')	קרה
	('קֵרָה', 'פועל', 'פיעל')	קרה
12. קְרִיאַת	['קְרִיאַת']
13. שְׁמַע	['שְׁמַע']
	('שָׁמַע', 'פועל', 'קל')	שמע
14. כְּנֶגְדָּן	['כְּנֶגְדָּן', 'נֶגְדָּן']
15. וְאַף	['וְאַף', 'אַף']
16. עַל	['עַל']
17. פִּי	['פִּי']
18. שֶׁאֵין	['שֶׁאֵין', 'אֵין']
	('אִיֵּן', 'פועל', 'פיעל')	אין
	('אִיֵּן', 'פועל', 'פיעל')	אין
19. בָּהֶן	['בָּהֶן']
20. כְּלוּם	['כְּלוּם', 'לוּם']
	('כִּלָּה', 'פועל', 'פ